# Central Bank Policy Rates - History vs. Forecast
© 2026 Jim Domeij

The chart shows quarterly policy rates from 2019 onward for the US (Fed), Eurozone (ECB), UK (BoE), and Japan (BoJ), extended with Bloomberg’s median economist forecasts through 2027. Solid line segments represent realized (actual) policy settings, while dashed segments show the current consensus expected path as of the data pull date. Comparing levels and slopes highlights where markets expect tightening, easing, or prolonged stability—and how the projected policy divergence/convergence evolves across regions.

In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/macro") / "cb-policy-rates-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.Chart(...)

In [2]:
# Get historical and forecast policy rates for FED and other central banks
from polars_bloomberg import BQuery

query = """
let(#policy_rate = policy_rate(pr=range(2019Q1, 2027Q4), pt=Q, act_est_data=AE);)
get(#policy_rate)
for(['US', 'EZ', 'GB', 'JP'])
preferences(
    dropCols=[
        'TICKER',
        'REVISION_DATE',
        'RATE_TYPE',
        'REPORTED_DATE',
        'CONCEPT',
        'ECO_SOURCE',
        'ACT_EST_SOURCE',
        'PERIOD',
        'PERIOD_END_DATE',
        'SEASONALITY'
    ]
)
"""

with BQuery() as bq:
    res = bq.bql(query)
df = res.combine()
df.head(3)

ID,#policy_rate,PERIOD_REFERENCE_DATE,AS_OF_DATE,ACT_EST_DATA
str,f64,date,date,str
"""US""",2.5,2019-03-31,2026-01-07,"""A"""
"""US""",2.5,2019-06-30,2026-01-07,"""A"""
"""US""",2.0,2019-09-30,2026-01-07,"""A"""


In [3]:
# Create chart
import altair as alt
import polars as pl

g_data = df.with_columns(
    pl.col("#policy_rate").mul(0.01).alias("pr"),
    pl.col("ID")
    .replace(
        {
            "US": "Fed (USA)",
            "EZ": "ECB (Eurozone)",
            "JP": "BoJ (Japan)",
            "GB": "BoE (UK)",
        }
    )
    .alias("ID"),
)
chart = (
    alt.Chart(g_data)
    .mark_line()
    .encode(
        x=alt.X("PERIOD_REFERENCE_DATE:T").title(None),
        y=alt.Y("pr:Q").axis(format="%").title("Policy Rate (%)"),
        color=alt.Color("ID:N")
        .title("Central Bank")
        .scale(domain=["Fed (USA)", "ECB (Eurozone)", "BoJ (Japan)", "BoE (UK)"]),
        strokeDash=alt.StrokeDash("ACT_EST_DATA:N").legend(None),
        size=alt.condition(
            alt.datum.ID == "Fed (USA)",
            alt.value(4),
            alt.value(2),
        ),
        tooltip=["ID:N", "PERIOD_REFERENCE_DATE:T", "#policy_rate:Q", "ACT_EST_DATA:N"],
    )
    .properties(
        width=600,
        height=400,
    )
)

# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)